In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import scipy.stats as ss, scipy.integrate as integrate

In [2]:
df = pd.read_csv('data/policecalls2022.csv')
df

,CDTS,EID,START_DATE,CALL_NUMBER,PRIORITY,REPORT_DATE,OFFENSE_DATE,OFFENSE_TIME,CALLTYPE_CODE,CALL_TYPE,FINAL_DISPO_CODE,FINAL_DISPO,COMMON_PLACE_NAME,ADDRESS,CITY,STATE
0,20220101042916PS,8913184,1/1/2022 12:00:00 AM,P220010180,2,1/1/2022 12:00:00 AM,1/1/2022 12:00:00 AM,02:49:02,1066,SUSPICIOUS PERSON,N,No report required; dispatch record only,NaN,[800]-[900] JEANNE AV,San Jose,CA
1,20220101043036PS,8913196,1/1/2022 12:00:00 AM,P220010191,2,1/1/2022 12:00:00 AM,1/1/2022 12:00:00 AM,03:05:51,415,DISTURBANCE,N,No report required; dispatch record only,NaN,[1500]-[1600] SOUTHWEST EX,San Jose,CA
2,20220101043827PS,8913252,1/1/2022 12:00:00 AM,P220010236,2,1/1/2022 12:00:00 AM,1/1/2022 12:00:00 AM,04:35:25,911UNK,UNK TYPE 911 CALL,CAN,Canceled,WESTMONT HS,[4800]-[4900] WESTMONT AV,San Jose,CA
3,20220101044026PS,8913216,1/1/2022 12:00:00 AM,P220010206,2,1/1/2022 12:00:00 AM,1/1/2022 12:00:00 AM,03:38:14,SUSCIR,SUSPICIOUS CIRCUMSTANCES,N,No report required; dispatch record only,NaN,[3600]-[3700] GREENLEE DR,San Jose,CA
4,20220102052501PS,8914182,1/1/2022 12:00:00 AM,P220010965,4,1/1/2022 12:00:00 AM,1/1/2022 12:00:00 AM,23:25:49,11300,NARCOTICS,A,Arrest Made,NaN,E CAPITOL EX & EVERGREEN PL,San Jose,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33037,20220108163647PS,8922559,1/9/2022 12:00:00 AM,P220080519,2,1/8/2022 12:00:00 AM,1/8/2022 12:00:00 AM,16:27:04,415A,"DISTURBANCE, FIGHT",N,No report required; dispatch record only,EASTRIDGE MALL,[2200]-[2200] EASTRIDGE LP,San Jose,CA
33038,20220108163747PS,8922442,1/9/2022 12:00:00 AM,P220080431,2,1/8/2022 12:00:00 AM,1/8/2022 12:00:00 AM,14:25:17,242,BATTERY,O,Supplemental report taken,NaN,[100]-[200] BRANHAM LN,San Jose,CA
33039,20220108164003PS,8922553,1/9/2022 12:00:00 AM,P220080513,2,1/8/2022 12:00:00 AM,1/8/2022 12:00:00 AM,16:22:08,415,DISTURBANCE,N,No report required; dispatch record only,NaN,[3300]-[3400] MT LOGAN DR,San Jose,CA
33040,20220108202855PS,8922752,1/9/2022 12:00:00 AM,P220080662,2,1/8/2022 12:00:00 AM,1/8/2022 12:00:00 AM,19:46:39,415,DISTURBANCE,G,Gone on Arrival/unable to locate,NaN,[1400]-[1500] KINGMAN AV,San Jose,CA


In [75]:
df_params = df[['OFFENSE_TIME', 'PRIORITY', 'CALLTYPE_CODE', 'FINAL_DISPO_CODE']]
df_params['FINAL_DISPO_CODE'] = df_params['FINAL_DISPO_CODE'].apply(
    lambda code: 1 if (code == 'A' or  code == 'B' or code == 'C') else 0)

arrests = df_params.loc[df_params['FINAL_DISPO_CODE'] == 1]
arrest_count = arrests.shape[0]
not_arrests = df_params.loc[df_params['FINAL_DISPO_CODE'] == 0]
not_arrest_count = not_arrests.shape[0]
arrest_mean, arrest_mode, arrest_std = arrests['PRIORITY'].mean(), arrests['PRIORITY'].mode()[0], arrests['PRIORITY'].std()
not_arrest_mean, not_arrest_mode, not_arrest_std = not_arrests['PRIORITY'].mean(), not_arrests['PRIORITY'].mode()[0], not_arrests['PRIORITY'].std()
print(f'''
    (where arrests is 'A' or 'B' or 'C')
    arrests:                        {arrest_count},
    not arrests:                 {not_arrest_count},
    max priority:                {df_params['PRIORITY'].max()}
    arrests priority mean: {arrest_mean}
    arrests priority mode: {arrest_mode}
    arrests priority std:      {arrest_std}
    not-arrests priority mean: {not_arrest_mean}
    not-arrests priority mode: {not_arrest_mode}
    not-arrests priority std:      {not_arrest_std}
''')
arrests


    (where arrests is 'A' or 'B' or 'C')
    arrests:                        964,
    not arrests:                 32078,
    max priority:                6
    arrests priority mean: 3.391078838174274
    arrests priority mode: 2
    arrests priority std:      1.6878908041018064
    not-arrests priority mean: 3.2649791134110604
    not-arrests priority mode: 3
    not-arrests priority std:      1.4072268319458991



/var/folders/5n/hy1r5gz51gjcr_ppq8r11n580000gn/T/ipykernel_11683/3663859867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_params['FINAL_DISPO_CODE'] = df_params['FINAL_DISPO_CODE'].apply(


,OFFENSE_TIME,PRIORITY,CALLTYPE_CODE,FINAL_DISPO_CODE
4,23:25:49,4,11300,1
127,04:17:30,2,415,1
169,10:55:07,1,1066W,1
259,13:06:09,3,415,1
285,14:46:48,4,11300,1
...,...,...,...,...
32905,02:55:03,6,1195,1
32913,09:15:58,2,6M,1
32948,09:33:31,3,594,1
32982,16:59:08,2,12020G,1


In [74]:
not_arrests

,OFFENSE_TIME,PRIORITY,CALLTYPE_CODE,FINAL_DISPO_CODE
0,02:49:02,2,1066,0
1,03:05:51,2,415,0
2,04:35:25,2,911UNK,0
3,03:38:14,2,SUSCIR,0
5,00:00:59,4,415FC,0
...,...,...,...,...
33036,16:31:38,3,1033A,0
33037,16:27:04,2,415A,0
33038,14:25:17,2,242,0
33039,16:22:08,2,415,0
